In [1]:
import pandas as pd
import osmnx as ox
import numpy as np

north = 55.9899
south = 55.8784
east = -3.0418
west = -3.3374

G = ox.load_graphml("data/edinburgh_graph.graphml")

In [2]:
def route_length(route):
    length = 0
    for i in range(len(route)-1):
        length += G[route[i]][route[i+1]][0]['length']
    return length/1000

def parsePath(pathStr):
    removeBrackets = pathStr[1:-1]
    split = removeBrackets.split('-')
    return [int(x) for x in split]

def mapit(node_ids_file, parsedPath):
    node_ids = np.loadtxt( "data/" + node_ids_file, delimiter=',', dtype=int)
    map_index_id = { k:v for (k,v) in zip(list(range(71)),node_ids)}
    mappedPath = [map_index_id[x] for x in parsedPath]

    route = ox.shortest_path(G, mappedPath[-1], mappedPath[0], weight='length')
    for i in range(len(mappedPath)-1):
        if ox.shortest_path(G, mappedPath[i], mappedPath[i+1], weight='length') == None:
            print("None")
            print(mappedPath[i], mappedPath[i+1])
        route += ox.shortest_path(G, mappedPath[i], mappedPath[i+1], weight='length')[1:]

    mapit = ox.plot_route_folium(G, route, route_color='red')

    import folium
    x = [G.nodes[map_index_id[i]]['x'] for i in range(len(node_ids))]
    y = [G.nodes[map_index_id[i]]['y'] for i in range(len(node_ids))]
    coords = list(zip(y, x))
    for location in coords:
        folium.Marker(location=location).add_to(mapit)

    print(route_length(route))

    mapit.save('temp_map.html')

    return mapit

In [6]:
def paths_sigmas(df_file):
    raw = pd.read_csv('../ACO/results/'+df_file, index_col=False)
    max_iters = raw['iteration'].max()
    max_iter_df = raw[raw['iteration'] == max_iters]
    sigma1 = max_iter_df[max_iter_df['sigma'] == 1]
    sigma20 = max_iter_df[max_iter_df['sigma'] == 20]

    best_sigma1 = sigma1[sigma1['bestLength'] == sigma1['bestLength'].min()]
    best_sigma20 = sigma20[sigma20['bestLength'] == sigma20['bestLength'].min()]

    best_sigma1_path = best_sigma1['bestPath'].values[0]
    best_sigma20_path = best_sigma20['bestPath'].values[0]

    parsed_sigma1 = parsePath(best_sigma1_path)
    parsed_sigma20 = parsePath(best_sigma20_path)

    return parsed_sigma1, parsed_sigma20

def path(df_file):
    raw = pd.read_csv('../ACO/results/'+df_file, index_col=False)
    max_iters = raw['iteration'].max()
    max_iter_df = raw[raw['iteration'] == max_iters]
    sigma20 = max_iter_df[max_iter_df['sigma'] == 20]
    best_sigma20 = sigma20[sigma20['bestLength'] == sigma20['bestLength'].min()]
    best_sigma20_path = best_sigma20['bestPath'].values[0]
    parsed_sigma20 = parsePath(best_sigma20_path)
    return parsed_sigma20

In [7]:
parsed_sigma20 = path('result_bins_normal_WS:_Food_Waste_Communal_City_Centre_nonplanar_71.csv')
mapit('bins_normal_WS:_Food_Waste_Communal_City_Centre_node_ids_71.csv', parsed_sigma20)

28.090412999999998


In [8]:
parsed_sigma20 = path('result_bins_normal_WS:_Food_Waste_Communal_City_Centre_euclidean_71.csv')
mapit('bins_normal_WS:_Food_Waste_Communal_City_Centre_node_ids_71.csv', parsed_sigma20)

35.47633600000001


In [9]:
parsed_sigma20 = path('result_bins_normal_WS:_Bottle_Bank_-_Clear_nonplanar_29.csv')
mapit('bins_normal_WS:_Bottle_Bank_-_Clear_node_ids_29.csv', parsed_sigma20)

71.02534999999983


In [10]:
parsed_sigma20 = path('result_bins_normal_WS:_Bottle_Bank_-_Clear_euclidean_29.csv')
mapit('bins_normal_WS:_Bottle_Bank_-_Clear_node_ids_29.csv', parsed_sigma20)

73.40506499999988
